In [74]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode

In [217]:


# Spotify Infos
scope = 'user-library-read'
username = "ilias.miraoui@gmail.com"

token = util.prompt_for_user_token(username, scope, client_id='6d6b00a4a500471e94c5a3f1559e8cc6', client_secret='500950b79db94ac88f6859d24e4e6033', redirect_uri='http://localhost:8888/callback')
if token:
    spotify = spotipy.Spotify(auth=token)
else:
    print("Connexion to spotify API failed.")
    sys.exit()

    


In [76]:
def simplify_string(s):
    return unidecode.unidecode(s.lower()).replace('the', ' ').replace(' ', '').translate(str.maketrans('', '', string.punctuation))


def similar(a, b, threshold=80):
    return fuzz.ratio(simplify_string(a), simplify_string(b)) >= threshold

In [105]:
def get_song_details(artist,track):
    song_details = dict()
    results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')
    maxi = None
    if results['tracks']['total'] > 0:
        for i in results['tracks']['items']:
            if similar(track, i['name'], 80) and similar(artist, i["artists"][0]["name"], 80):
                maxi = i
                break
        if maxi is None:
            return None
        if results is None or len(results) == 0:
            return None
        _id = maxi["id"] 
        song_details[_id] = dict()
        song_details[_id]["explicit"] = maxi["explicit"]
        song_details[_id]["duration_ms"] = maxi["duration_ms"]
        song_details[_id]["disc_number"] = maxi["disc_number"]
        song_details[_id]["track_number"] = maxi["track_number"]
    return song_details


    

In [201]:
def get_song_features(_id):
    song_features = dict()
    results = spotify.audio_features(_id)
    results = results[0]
    if results != None: 
        song_features["danceability"] = results["danceability"]
        song_features["energy"] = results["energy"]
        song_features["key"] = results["key"]
        song_features["loudness"] = results["loudness"]
        song_features["mode"] = results["mode"]
        song_features["speechiness"] = results["speechiness"]
        song_features["acousticness"] = results["acousticness"]
        song_features["instrumentalness"] = results["instrumentalness"]
        song_features["liveness"] = results["liveness"]
        song_features["valence"] = results["valence"]
        song_features["tempo"] = results["tempo"]
        song_features["time_signature"] = results["time_signature"]
        return song_features
    else:
        return None

In [79]:
song = get_song_details("Jimi Hendrix","Purple Haze")
features = get_song_features(song.keys())

In [80]:
song.keys()

dict_keys(['0wJoRiX5K5BxlqZTolB2LD'])

In [210]:
with open('hot-100_data.pkl', 'rb') as f:
        df = pickle.load(f)
with open('hot-100_chart.pkl', 'rb') as f:
        chart = pickle.load(f)

In [177]:
with open('song_df.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [215]:
song_df = df.drop_duplicates(["artist","title"])
song_df["spotify_id"] = None

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [226]:
i=0
for song in song_df.index:
    if pd.isna(song_df.loc[song,"spotify_id"]) == True:
        title = song_df.loc[song,"title"]
        artist = song_df.loc[song,"artist"]
        song_details = get_song_details(artist,title)
        if type(song_details) == dict and len(song_details)>0 :
            song_df.loc[song,"spotify_id"] = list(song_details.keys())[0]
            for feature in list(song_details[list(song_details.keys())[0]].keys()):
                song_df.loc[song,"spotify_"+feature] = song_details[list(song_details.keys())[0]][feature]
        else:
            i = i+1
            print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [231]:
with open('song_df_100.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [228]:
song_df["spotify_key"] = None

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [229]:
for song in song_df[song_df["spotify_id"].notnull()].index:
    spotify_key = song_df.loc[song,"spotify_key"]
    spotify_id = song_df.loc[song,"spotify_id"]
    if pd.isna(spotify_key) == True:
        song_features = get_song_features(spotify_id)
        if song_features != None:
            for feature in list(song_features.keys()):
                song_df.loc[song,"spotify_"+feature] = song_features[feature]

In [230]:
song_df.dropna()

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_energy,spotify_loudness,spotify_mode,spotify_speechiness,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.405,-5.679,1.0,0.0319,0.7510,0.000000,0.1050,0.4460,109.891,4.0
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.762,-3.497,1.0,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,4.0
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.624,-3.046,0.0,0.1140,0.1100,0.000000,0.1230,0.4120,158.087,4.0
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.340,-9.005,1.0,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,4.0
5,Lizzo,Good As Hell,False,6,2019-11-16,3Yh9lZcWyKrK9GjbhuS0hR,False,159735.0,1.0,1.0,...,0.919,-3.180,0.0,0.0875,0.2560,0.000000,0.4420,0.5350,95.982,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113099,The Rolling Stones,Saint Of Me,False,100,1998-03-28,6OEpSYq8wdqnl01hsoQGQY,False,314626.0,1.0,8.0,...,0.918,-5.775,1.0,0.0504,0.0049,0.044300,0.0895,0.4940,121.541,4.0
113147,Nu Flavor,Heaven,False,48,1998-03-21,0zLMsK6UeAP4KrbekFbHvC,False,298133.0,1.0,3.0,...,0.406,-12.100,1.0,0.0235,0.8440,0.000176,0.0875,0.3500,96.178,4.0
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.993,-5.067,1.0,0.0464,0.0105,0.034100,0.4030,0.7020,124.016,4.0
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.454,-8.528,1.0,0.0253,0.4320,0.001680,0.0446,0.5450,79.913,4.0


In [224]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,None,NaN,NaN,NaN,NaN
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0
113194,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,None,NaN,NaN,NaN,NaN
113198,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,None,NaN,NaN,NaN,NaN
